# Welcome to the classroom-scale thermal comfort mapping with Energy Plus

What to advise a local authoritie to reduce indoor heat stress in school classrooms?
- When are the hottest periods—both seasonally and throughout the day?
- What existing features contribute to worsening the current thermal sensation?
- How much thermal comfort can be achieved through passive strategies?
- What are the key passive design strategies to prioritize?

To answer these questions, we invite you to dive into the Energy Plus thermal model:
* Understand the geometry and its relationship with the sun.
* Explore the window openings,their size, characteristics and opening schedules
* Explore the different thermal values in the materials and constructions in the building envelope.
* Make sense of the output, its patterns, and what it reveals

But ... don't feel limited by what we've provided!

Feel free to bring in your own tools, datasets, and creative approaches ...whatever you feel comfortable with or curious about. Hackathons are meant for exploration, not constraint.

Enjoy!
---

# Setting up what we need...

* Mount the Google Drive, to access files and datasets
* Install the required packages and tools
* Set up directory and filenames (**some need to be adjusted**)

In [ ]:
# Some Initial imports
import os
import tarfile
import shutil
import importlib
import glob
import subprocess

In [ ]:
# Mount Google Drive, to access the datasets and write your results.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# ===> SETTINGS TO ADJUST <====

# Your name - refers to the Google Drive folder name
YOUR_NAME = "Matthias"

# city of interest: "Davao" or "Timbuktu"
# city = "Davao"
city = "Timbuktu"

# the simulation's run name
  # => By default, this is set to baseline, representing current conditions in the IDF file.

  # => Alternative scenarios can be done as well, though the procedure is not automated in this notebook.
  # We advise you to reach out to the support team in case you want to try something ...
scenario_name = "baseline"

# Your own baseline directory - check where you have mounted the ICUC12_Hackathon folder
BASE_DIR = os.path.join('/content/drive/MyDrive/ICUC12_Hackathon')
print('> BASE_DIR: ',BASE_DIR)

In [ ]:
# ===> SETTINGS NOT TO CHANGE <====

# Setting files and directories, based on personal folder, city, and scenario
model = "ENERGYPLUS"

# Create a link to your personal directory
PERSONAL_FOLDER = os.path.join(BASE_DIR, YOUR_NAME)
print('> PERSONAL_FOLDER: ', PERSONAL_FOLDER)

# Figure folder, to store all your nice visuals
FIGURE_FOLDER = os.path.join(PERSONAL_FOLDER, "figures")
print('> FIGURE_FOLDER: ', FIGURE_FOLDER)

# Create link to shared directory
SHARED_DIR = os.path.join(BASE_DIR, "SHARED_FOLDER")
print('> SHARED_DIR: ',SHARED_DIR)

# Path to shared notebooks
SHARED_FUNCTIONS = os.path.join(SHARED_DIR, "functions")
print(f'> SHARED FUNCTIONS FOLDER: ',SHARED_FUNCTIONS)

# Set MODEL INPUT DATA folder, is the shared data folder
MODEL_INPUT_FOLDER = os.path.join(SHARED_DIR, "data", city, model, scenario_name)
print(f'> {model} INPUT FOLDER: ', MODEL_INPUT_FOLDER)

# Set MODEL OUTPUT folder, under your own personal directory
MODEL_OUTPUT_FOLDER = os.path.join(PERSONAL_FOLDER, city, model, scenario_name)
print(f'> {model} OUTPUT FOLDER: ', MODEL_OUTPUT_FOLDER)

#Set MODELS_folder in personal directory
MODELS_FOLDER = os.path.join(PERSONAL_FOLDER, "models")
print(f'> MODELS FOLDER: ', MODELS_FOLDER)

# Create some folders that might not exist
for folder in [
    MODEL_OUTPUT_FOLDER,
    FIGURE_FOLDER,
    MODELS_FOLDER,
    ]:
  if not os.path.exists(folder):
    os.makedirs(folder)

### **Installing EnergyPlus 25.1.0 from Google Drive to Google Drive**

Each time a Google Colab session is launched, it runs on a new virtual machine. As a result, the EnergyPlus installation must be validated at the start of every session.

These instructions assume that a shortcut to the shared **ICUC12_Hackathon** folder has already been created in your personal Google Drive. This shortcut is necessary to access the required files.

Please also note that whenever a Google Drive folder is mounted, you'll need to grant access permissions again.


In [ ]:
# Install a bunch of packages - with "-q" option to reduce verbose output
!pip install -q import_ipynb

In [ ]:
import import_ipynb
os.chdir(SHARED_FUNCTIONS)

# import all functions made for this course
import energyplus_functions
importlib.reload(energyplus_functions)

from energyplus_functions import install_energyplus
from energyplus_functions import run_energyplus
from energyplus_functions import add_comfort_assessment_to_csv
from energyplus_functions import add_drybulb_to_comfort
from energyplus_functions import plot_heatmap
from energyplus_functions import plot_overheating_assessment
from energyplus_functions import plot_hottest_week_of_the_year

In [ ]:
 # Install the Energylus model -- this can take a bit of time
 install_energyplus(SHARED_DIR, MODELS_FOLDER)

# Simulate with EnergyPlus



### Running the Simulations

To run EnergyPlus in your personal folder, the following things are done within the run_energyplus function:
- Copy all EnergyPlus .idf files into the **IDF folder**.
- Copy the necesary weather files into the **EPW folder**.
- The script automatically detects the .idf files, runs the simulations, and generates results within a new folder inside the **MODEL_OUTPUT_FOLDER**.

In [ ]:
run_energyplus(MODEL_INPUT_FOLDER, MODEL_OUTPUT_FOLDER)

### Generating a .csv file with hourly results

After the simulations are completed, the final step is to generate a CSV file from the EnergyPlus Standard Output file (.ESO). This requires using a tool called ReadVarsESO, which is located within the EnergyPlus installation directory.


In [ ]:
# --- Define path ---
readvars_path = os.path.join(MODELS_FOLDER, 'EnergyPlus-25.1.0-68a4a7c774-Linux-Ubuntu22.04-x86_64/PostProcess/ReadVarsESO')

# --- 3. Validate ReadVarsESO executable ---
print("\nVerifying ReadVarsESO...")
if not os.path.isfile(readvars_path):
    raise FileNotFoundError(f"❌ ReadVarsESO not found at: {readvars_path}")
print(f"✅ Found ReadVarsESO at: {readvars_path}")

# --- Process the single .eso file in MODEL_OUTPUT_FOLDER ---
print(f"\nSearching for .eso file in: {MODEL_OUTPUT_FOLDER}...")

# Search directly in MODEL_OUTPUT_FOLDER for .eso files
eso_files = glob.glob(os.path.join(MODEL_OUTPUT_FOLDER, '*.eso'))

if not eso_files:
    print(f"⚠️ No .eso file found in: {MODEL_OUTPUT_FOLDER}")
else:
    # Assuming there's only one, take the first one found
    eso_file = eso_files[0]
    print(f"Found .eso file: {os.path.basename(eso_file)}")
    print(f"Processing directory: {MODEL_OUTPUT_FOLDER}")

    try:
        original_cwd = os.getcwd()
        os.chdir(MODEL_OUTPUT_FOLDER)  # ReadVarsESO writes output to current dir

        readvars_process = subprocess.run([readvars_path], capture_output=True, text=True)

        os.chdir(original_cwd) # Always change back to original CWD

        if readvars_process.returncode == 0:
            print(f"✅ Successfully created eplusout.csv in: {MODEL_OUTPUT_FOLDER}")
        else:
            print(f"❌ ERROR: ReadVarsESO failed in {MODEL_OUTPUT_FOLDER}")
            print("--- STDERR ---")
            print(readvars_process.stderr)

    except Exception as e:
        print(f"❌ Exception occurred while processing {MODEL_OUTPUT_FOLDER}: {e}")
        # Ensure we change back to original_cwd even if an exception occurs before os.chdir(original_cwd) is reached
        if 'original_cwd' in locals():
            os.chdir(original_cwd)

print("\n🎯 .eso file processing complete.")

# Analysis of results

## Comfort Assessment Using Simulation Results

Once the simulations have been run, each `.IDF` file generates a dedicated results folder located inside `/scenario_name/`.

In the previous step, a CSV file containing air and operative temperatures (in degrees Celsius) was produced. To evaluate thermal comfort, these temperatures must be compared against pre-calculated comfort benchmarks.

Within the `/EPW/` folder, there is a CSV file containing the prevailing and thermal neutrality temperatures already extracted from the EPW file.

The script then generates an additional CSV file inside the `/analysis/` folder, adding new columns to help determine whether the operative temperature falls within the comfort range. The columns added are:

- **Yes_No_Overheating**: A value of `0` indicates that the indoor temperature is within the comfort range. A value of `1` denotes overheating.
- **Distance_from_comfort**: Indicates the number of degrees above the upper comfort limit, showing how much the indoor environment exceeds acceptable thermal conditions.
- **Hours_1°C_From_Comfort**: Number of hours where the operative temperature is 1°C above the comfort limit. Thermal stress is barely noticeable and may only affect sensitive individuals.
- **Hours_2°C_From_Comfort**: Number of hours where the operative temperature is 2°C above the comfort limit. Thermal stress becomes noticeable for most occupants, producing a slightly warm sensation. However, adaptive opportunities can still mitigate discomfort.
- **Hours_3°C_From_Comfort**: Number of hours where the operative temperature is 3°C above the comfort limit. Thermal stress is clearly noticeable, producing a hot sensation. Adaptive opportunities remain effective in restoring comfort.
- **Hours_4°C_From_Comfort**: Number of hours where the operative temperature is 4°C above the comfort limit. Thermal stress is very noticeable, producing a very hot sensation. Conditions above this threshold may pose health risks for both sensitive and non-sensitive occupants.



In [ ]:
add_comfort_assessment_to_csv(city, MODEL_INPUT_FOLDER, MODEL_OUTPUT_FOLDER)

## Adding the Dry Bulb Temperature from the weather file to the Comfort Analysis

Using the weather file as the data source, we will extract the hourly dry-bulb temperature values to serve as a reference during the comfort assessment.



In [ ]:
add_drybulb_to_comfort(MODEL_INPUT_FOLDER, MODEL_OUTPUT_FOLDER)

# Data visualization


### Heatmaps

Creating heatmaps to visualize the results.

In [ ]:
figname = os.path.join(FIGURE_FOLDER, f"{model}_{city}_{scenario_name}_heatmaps.png")
plot_heatmap(MODEL_OUTPUT_FOLDER, figname)

### Overheating Assessment

The overheating assessment consists of quantifying both the number of hours above the comfort benchmark and the extent to which those hours exceed the comfort threshold.


In [ ]:
#Scenarios you want to analyse
scenarios=['baseline'] #add more like ['baseline', 'scenario1', ...]

figname = os.path.join(FIGURE_FOLDER, f"{model}_{city}_overheating.png")
plot_overheating_assessment(MODEL_OUTPUT_FOLDER, figname, scenarios)

### What happens during heatwave?


Here, we make a deep-dive into the results for the selected heatwave period that is also use by TARGET and SOLWEIG.

The script is designed to detect multiple result files from different simulation runs and generate a comparative graph to visualize their performance during this critical week.

In [ ]:
#Scenarios you want to analyse
scenarios=['baseline'] #add more like ['baseline', 'scenario1', ...]

plot_hottest_week_of_the_year(MODEL_OUTPUT_FOLDER, figname, scenarios)